In [1]:
# # Mount GDRIVE jika menggunakan Google Collab
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# PATH = '/content/drive/MyDrive/Tugas Akhir - Iki/TA2/Source Code/program'
# PATH_1 = '/backend_api/data/topic classification data/'
PATH = ''
PATH_1 = './backend_api/data/topic classification data/'

In [3]:
# !pip install transformers

In [4]:
import pandas as pd

data = pd.read_csv(PATH + PATH_1 + 'topic_classification_balanced.csv', delimiter=';')
data

,kategori,isi_preprocessed
0,hot,cerai hidup kacau cerita hijrah peggy melati s...
1,ekonomi,orang kaya orang kaya beneran
2,kesehatan,sebar positif virus corona indonesia
3,hot,cerita mawar jongh hadap serang netizen
4,hot,grande blackpink muncul lady gaga
...,...,...
28652,teknologi,browser vivaldi sedia ios bawa fitur fokus pri...
28653,teknologi,meta headset generasi pasang kacamata pintar s...
28654,teknologi,samsung geser jendela seri galaxy februari jan...
28655,teknologi,dyson floorcare tech letak inovasi desain rama...


In [5]:
data["kategori"].value_counts()

kategori
hot                   6000
ekonomi               6000
berita dan politik    6000
kesehatan             4983
teknologi             2993
olahraga              2681
Name: count, dtype: int64

In [6]:
data = data.dropna().reset_index(drop=True)

In [7]:
data.isnull().sum()

kategori            0
isi_preprocessed    0
dtype: int64

In [8]:
data.columns = ['label', 'text']
data

,label,text
0,hot,cerai hidup kacau cerita hijrah peggy melati s...
1,ekonomi,orang kaya orang kaya beneran
2,kesehatan,sebar positif virus corona indonesia
3,hot,cerita mawar jongh hadap serang netizen
4,hot,grande blackpink muncul lady gaga
...,...,...
28643,teknologi,browser vivaldi sedia ios bawa fitur fokus pri...
28644,teknologi,meta headset generasi pasang kacamata pintar s...
28645,teknologi,samsung geser jendela seri galaxy februari jan...
28646,teknologi,dyson floorcare tech letak inovasi desain rama...


In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
data

,label,text
0,2,cerai hidup kacau cerita hijrah peggy melati s...
1,1,orang kaya orang kaya beneran
2,3,sebar positif virus corona indonesia
3,2,cerita mawar jongh hadap serang netizen
4,2,grande blackpink muncul lady gaga
...,...,...
28643,5,browser vivaldi sedia ios bawa fitur fokus pri...
28644,5,meta headset generasi pasang kacamata pintar s...
28645,5,samsung geser jendela seri galaxy februari jan...
28646,5,dyson floorcare tech letak inovasi desain rama...


In [10]:
le.inverse_transform([0,1,2,3,4,5])

array(['berita dan politik', 'ekonomi', 'hot', 'kesehatan', 'olahraga',
       'teknologi'], dtype=object)

In [11]:
data['text'] = data['text'].astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28648 entries, 0 to 28647
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   28648 non-null  int32 
 1   text    28648 non-null  object
dtypes: int32(1), object(1)
memory usage: 335.8+ KB


# Training Model

In [12]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
N_CLASSES = 3

## Make dataset

In [13]:
from datasets import Dataset

d:\Kuliah\Rifky - Full program setelah revisi\virtualenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 28648
})

In [15]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset['train']
val_test_dataset  = dataset['test']
val_test_dataset = val_test_dataset.train_test_split(test_size=0.2, seed=42)
val_dataset, test_dataset = val_test_dataset['train'], val_test_dataset['test']
train_dataset, val_dataset, test_dataset

(Dataset({
     features: ['label', 'text'],
     num_rows: 22918
 }),
 Dataset({
     features: ['label', 'text'],
     num_rows: 4584
 }),
 Dataset({
     features: ['label', 'text'],
     num_rows: 1146
 }))

## Training

In [16]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = "cahya/bert-base-indonesian-1.5G"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=6)  # num_labels sesuai jumlah kelas

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def tokenize(batch):
    return tokenizer(batch['text'], max_length=MAX_LEN, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 4584/4584 [00:00<00:00, 5959.75 examples/s]


In [18]:
class_weights = (1 - (data["label"].value_counts().sort_index() / len(data))).values
class_weights

array([0.79063111, 0.79063111, 0.79063111, 0.82613097, 0.90645071,
       0.89552499])

In [19]:
import torch

class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.7906, 0.7906, 0.7906, 0.8261, 0.9065, 0.8955], device='cuda:0')

In [20]:
try:
  train_dataset = train_dataset.rename_column("label", "labels")
  val_dataset = val_dataset.rename_column("label", "labels")
except Exception as e:
  print(e)

val_dataset, train_dataset

(Dataset({
     features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 4584
 }),
 Dataset({
     features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 22918
 }))

In [21]:
from transformers import Trainer, TrainingArguments
from torch import nn

In [22]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def compute_metrics(pred):
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    # acc = accuracy_score(labels, preds)
    # f1 = f1_score(labels, preds, average='weighted')
    # return {
    #     'accuracy': acc,
    #     'f1': f1,
    # }
    labels = pred.label_ids
    if isinstance(pred.predictions, tuple):
        preds_logits = pred.predictions[0]
    else:
        preds_logits = pred.predictions

    # Apply argmax to the logits to get the predicted class indices
    preds = preds_logits.argmax(-1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'f1': f1,
    }

In [23]:
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs.get("labels")
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"

# training_args = TrainingArguments(
#     eval_strategy="epoch",  # must evaluate to compute metrics
#     logging_strategy="epoch",     # must log during evaluation
#     logging_dir="./logs",
#     logging_steps=100,
#     save_strategy="epoch",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=4,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     output_dir=PATH+"/my_indodistilbert_topic_classification",
#     overwrite_output_dir=True,
#     fp16=True
# )

training_args = TrainingArguments(
    output_dir=PATH+"./my_indobert3_topic_classification",
    eval_strategy="epoch",  # must evaluate to compute metrics
    logging_strategy="epoch",     # must log during evaluation
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    learning_rate=2e-5,
)


# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
C:\Users\Kocheng\AppData\Local\Temp\ipykernel_108680\78627794.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedLossTrainer(


In [25]:
# metrics = trainer.evaluate()
# print(metrics)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.562200,0.423722,0.854930,0.855770
2,0.318900,0.394177,0.874564,0.874272
3,0.207600,0.469712,0.877836,0.877480
4,0.134100,0.502870,0.880017,0.879900


TrainOutput(global_step=5732, training_loss=0.30568058309401985, metrics={'train_runtime': 655.2207, 'train_samples_per_second': 139.91, 'train_steps_per_second': 8.748, 'total_flos': 2516754823419384.0, 'train_loss': 0.30568058309401985, 'epoch': 4.0})

In [27]:
model.save_pretrained(PATH+"./my_indobert3_topic_classification")
tokenizer.save_pretrained(PATH+"./my_indobert3_topic_classification")

('./my_indobert3_topic_classification\\tokenizer_config.json',
 './my_indobert3_topic_classification\\special_tokens_map.json',
 './my_indobert3_topic_classification\\vocab.txt',
 './my_indobert3_topic_classification\\added_tokens.json')

In [28]:
import pandas as pd
import torch

# tokenized_input = tokenizer(test_dataset['text'], return_tensors="pt", truncation=True, padding=True)
# tokenized_input
test_dataset_tokenized = test_dataset.map(tokenize, batched=True)
test_dataset_tokenized

Map: 100%|██████████| 1146/1146 [00:00<00:00, 6194.64 examples/s]


Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1146
})

In [29]:
prediction = trainer.predict(test_dataset_tokenized)
prediction

PredictionOutput(predictions=array([[-0.03906064, -2.122778  ,  5.0910273 ,  1.0997798 , -1.127695  ,
        -3.2887676 ],
       [-0.35802788, -1.0590193 , -1.9386947 , -1.5658906 ,  6.1707506 ,
        -0.9741962 ],
       [-0.18988557, -0.71156806, -2.008615  , -0.9278082 , -1.1775438 ,
         5.3751283 ],
       ...,
       [-1.0907438 , -2.2357595 ,  5.8314915 , -0.9230313 , -0.37687102,
        -1.3329017 ],
       [ 0.4346623 , -1.7938585 , -2.734862  ,  5.7809043 , -1.6647055 ,
        -1.1945399 ],
       [-0.6828432 , -1.7143351 ,  6.2460165 , -0.8048479 , -1.2123233 ,
        -2.2290936 ]], dtype=float32), label_ids=array([2, 4, 5, ..., 2, 3, 2], dtype=int64), metrics={'test_loss': 0.47351112961769104, 'test_accuracy': 0.8542757417102966, 'test_f1': 0.8534614416117362, 'test_runtime': 1.9495, 'test_samples_per_second': 587.832, 'test_steps_per_second': 36.932})

In [30]:
compute_metrics(prediction)

{'accuracy': 0.8542757417102966, 'f1': 0.8534614416117362}

# Load Model for Testing

In [ ]:
from transformers import AutoConfig, AutoModel, AutoModelForSequenceClassification

OUTPUT_PATH = PATH + "./my_indobert3_topic_classification"

# config = AutoConfig.from_pretrained(OUTPUT_PATH + "config.json")
model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_PATH)
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import AutoTokenizer, BertTokenizer

# tokenizer = AutoTokenizer.from_pretrained(OUTPUT_PATH)
# tokenizer
model_name = "cahya/bert-base-indonesian-1.5G"
tokenizer = BertTokenizer.from_pretrained(model_name)